In [ ]:
# @title
# https://medium.com/georgian-impact-blog/the-practical-guide-to-llms-flan-t5-6d26cc5f14c0

#https://github.com/pedro-r-marques/tutorial-t5-fine-tune

In [ ]:
# @title
USE_SAME_FEW_SHOTS=True

# INSTALLAZIONE E IMPORT LIBRERIE

In [ ]:
# @title
import pickle

In [ ]:
# @title
!pip install -q pillow==10.0.1 --progress-bar off
!pip install -q langchain[llms] --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.2/324.2 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# @title
#!pip install --upgrade pip
!pip install -q transformers --progress-bar off # Download e caricamento modelli
!pip install -q bitsandbytes --progress-bar off # Quantizzazione
!pip install -q accelerate --progress-bar off
!pip install -q sentencepiece --progress-bar off

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.6 MB/s eta 0:00:00


In [ ]:
# @title
from google.colab import drive
from IPython.display import Image, display

mount_point = "/content/gdrive"
base_path = mount_point + "/MyDrive/Colab/SDG/data"
input_data_path = base_path + "/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx"
input_data_path_train = base_path + "/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx"

output_data_path = base_path + "/../Flan-T5/data/output/2023.11.22-8_T5_run_completo-xxlarge_zero-shot-REBOOT"

drive.mount(mount_point, force_remount=True)
# data_path = '/content/gdrive/MyDrive/Colab/SDG/Data/input/2023.10.17-1_GPT_zero-shot-learning_all-17-sdg.xlsx'

Mounted at /content/gdrive


In [ ]:
# @title
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# @title
# T5 comes in different sizes:

#     t5-small
#     t5-base
#     t5-large (800M di parametri)
#     t5-3b (flan-t5-xl)
#     t5-11b.(flan-t5-xxl)

In [ ]:
# @title
# model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "meta-llama/Llama-2-13b-chat-hf"
# model_id = "google/flan-t5-small" # 60M
# model_id = "google/flan-t5-base" # 220M
# model_id = "google/flan-t5-large" # 800M
# model_id = "google/flan-t5-xl" # 3B
model_id = "google/flan-t5-xxl" # 11B

In [ ]:
# @title
# nbit = 4
# # llamasize = '7B'
# llamasize = '13B'
# temp = 0.1
# outdir = 'data/output/2023.11.13-1_T5_run_completo-large___TEST/'
# outdir = 'data/output/2023.11.22-6_T5_run_completo-base_zero-shot-REBOOT/'
# outfilename = outdir+f'sdg_classification_T5_1020-texts_temp-{temp}_nbit-{nbit}_llamasize-{llamasize}.xlsx'
outfilename = output_data_path+f'/sdg_classification_T5_1020-texts.xlsx'

outfilename

'/content/gdrive/MyDrive/Colab/SDG/data/../Flan-T5/data/output/2023.11.22-8_T5_run_completo-xxlarge_zero-shot-REBOOT/sdg_classification_T5_1020-texts.xlsx'

In [ ]:
# @title
# data = pd.read_excel("data/input/data_sel_4_few_shot_examples.xlsx")
data = pd.read_excel(input_data_path)
# data = data.drop(columns=['Unnamed: 0', 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
# data.iloc[61] #text

In [ ]:
# @title
# CUDA

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(model_id)
model = T5ForConditionalGeneration.from_pretrained(model_id, device_map="auto") #, torch_dtype=torch.float16, load_in_4bit=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00005.safetensors:   0%|          | 0.00/9.96G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/6.06G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# @title
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

In [ ]:
# @title
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig # Questi moduli sono dedicati al modello LLama2
from transformers import AutoModelForCausalLM, AutoTokenizer # Questi moduli permettono di scaricare qualsiasi modello e tokenizer
import pandas as pd
import torch # PyTorch
import langchain

In [ ]:
# @title
###torch.cuda.empty_cache()

## TEST

In [ ]:
# @title
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) #|
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel(input_data_path_train).drop(columns=['Unnamed: 0']).sample(5)

# data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.8125
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.0000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.0000


In [ ]:
# @title
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Class name: SDG "+str(row['sdg'])+'\n\n'

    few_shots += input_text
    few_shots += output_text

# print(few_shots)

Input text: "This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
Class name: SDG 12

Input text: "The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against t

In [ ]:
# @title
# FEW_SHOT_3 = """Text: This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain.
# Question: What is this text about?
# Answer: N (Ensure sustainable consumption and production patterns)


# Text: The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism.
# Question: What is this text about?
# Answer: R (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels)


# Text: As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far.
# Question: What is this text about?
# Answer: H (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all)

# """

In [ ]:
# @title
# FEW_SHOT_3 = """Text: This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain.
# Question: What is this text about?
# Answer: SDG 12 (Ensure sustainable consumption and production patterns)


# Text: The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism.
# Question: What is this text about?
# Answer: SDG 16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels)


# Text: As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far.
# Question: What is this text about?
# Answer: SDG 8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all)


# """

In [ ]:
# @title
FEW_SHOT_3 = """Text: This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain.
Class: SDG 12 (Ensure sustainable consumption and production patterns)

Text: The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism.
Class: SDG 16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels)

Text: As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far.
Class SDG 8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all)


"""

In [ ]:
# @title
data_sel = data
# data_sel = data.sample(10)

In [ ]:
# @title
# options_ = """A: End poverty in all its forms everywhere.
# B: End hunger, achieve food security and improved nutrition and promote sustainable agriculture.
# C: Ensure healthy lives and promote well-being for all at all ages.
# D: Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all.
# E: Achieve gender equality and empower all women and girls.
# F: Ensure availability and sustainable management of water and sanitation for all.
# G: Ensure access to affordable, reliable, sustainable and modern energy for all.
# H: Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all.
# I: Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation.
# L: Reduce inequality within and among countries.
# M: Make cities and human settlements inclusive, safe, resilient and sustainable.
# N: Ensure sustainable consumption and production patterns.
# O: Take urgent action to combat climate change and its impacts.
# P: Conserve and sustainably use the oceans, seas and marine resources for sustainable development.
# Q: Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss.
# R: Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels.
# S: Other

# """

# options_

In [ ]:
# @title
options_ = "SDG_1 (End poverty in all its forms everywhere), \
SDG_2 (End hunger, achieve food security and improved nutrition and promote sustainable agriculture), \
SDG_3 (Ensure healthy lives and promote well-being for all at all ages), \
SDG_4 (Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all), \
SDG_5 (Achieve gender equality and empower all women and girls), \
SDG_6 (Ensure availability and sustainable management of water and sanitation for all), \
SDG_7 (Ensure access to affordable, reliable, sustainable and modern energy for all), \
SDG_8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all), \
SDG_9 (Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation), \
SDG_10 (Reduce inequality within and among countries), \
SDG_11 (Make cities and human settlements inclusive, safe, resilient and sustainable), \
SDG_12 (Ensure sustainable consumption and production patterns), \
SDG_13 (Take urgent action to combat climate change and its impacts), \
SDG_14 (Conserve and sustainably use the oceans, seas and marine resources for sustainable development), \
SDG_15 (Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss), \
SDG_16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels), \
SDG_0 (Other)."

In [ ]:
# @title
# options_ = "A (End poverty in all its forms everywhere), \
# B (End hunger, achieve food security and improved nutrition and promote sustainable agriculture), \
# C (Ensure healthy lives and promote well-being for all at all ages), \
# D (Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all), \
# E (Achieve gender equality and empower all women and girls), \
# F (Ensure availability and sustainable management of water and sanitation for all), \
# G (Ensure access to affordable, reliable, sustainable and modern energy for all), \
# H (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all), \
# I (Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation), \
# L (Reduce inequality within and among countries), \
# M (Make cities and human settlements inclusive, safe, resilient and sustainable), \
# N (Ensure sustainable consumption and production patterns), \
# O (Take urgent action to combat climate change and its impacts), \
# P (Conserve and sustainably use the oceans, seas and marine resources for sustainable development), \
# Q (Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss), \
# R (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels), \
# S (Other).

# "

In [ ]:
# @title
# # vedi https://www.ibm.com/docs/en/watsonx-as-a-service?topic=lab-sample-prompts#sample1

# template_part1 = """Classify a text into one of 17 classes: SDG 1, SDG 2, SDG 3, SDG 4, SDG 5, SDG 6, SDG 7, SDG 8, SDG 9, SDG 10, SDG 11, SDG 12, SDG 13, SDG 14, SDG 15, SDG 16, SDG 0.

# Class name: SDG 1
# Description: A text about ending poverty in all its forms everywhere

# Class name: SDG 2
# Description: A text about ending hunger, achieving food security and improved nutrition, and promoting sustainable agriculture.

# Class name: SDG 3
# Description: A text about ensuring healthy lives and promoting well-being for all at all ages.

# Class name: SDG 4
# Description: A text about ensuring inclusive and equitable quality education and promoting lifelong learning opportunities for all.

# Class name: SDG 5
# Description: A text about achieving gender equality and empowering all women and girls.

# Class name: SDG 6
# Description: A text about ensuring the availability and sustainable management of water and sanitation for all.

# Class name: SDG 7
# Description: A text about ensuring access to affordable, reliable, sustainable, and modern energy for all.

# Class name: SDG 8
# Description: A text about promoting sustained, inclusive, and sustainable economic growth, full and productive employment, and decent work for all.

# Class name: SDG 9
# Description: A text about building resilient infrastructure, promoting inclusive and sustainable industrialization, and fostering innovation.

# Class name: SDG 10
# Description: A text about reducing inequality within and among countries.

# Class name: SDG 11
# Description: A text about making cities and human settlements inclusive, safe, resilient, and sustainable.

# Class name: SDG 12
# Description: A text about ensuring sustainable consumption and production patterns.

# Class name: SDG 13
# Description: A text about taking urgent action to combat climate change and its impacts.

# Class name: SDG 14
# Description: A text about conserving and sustainably using the oceans, seas, and marine resources for sustainable development.

# Class name: SDG 15
# Description: A text about protecting, restoring, and promoting sustainable use of terrestrial ecosystems, sustainably managing forests, combating desertification, halting and reversing land degradation, and halting biodiversity loss.

# Class name: SDG 16
# Description: A text about promoting peaceful and inclusive societies for sustainable development, providing access to justice for all, and building effective, accountable, and inclusive institutions at all levels.

# Class name: SDG 0
# Description: A text about any other topic different respect the previous ones.
# \n\n
# """+few_shots+"""

# Input text: """#\"{text}\"
# # Class name: """

In [ ]:
# @title
# print(template)

In [ ]:
# @title
# https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py (riga 578)
# "{title}\n\n{text}\n\nMulti-choice problem: What is this text about?\n{options_}"

In [ ]:
# @title
# # https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py (riga 2193)

# # """Patterns for few-shot tasks.

# # The few-shot input are composed by a few examplers followed by final_suffix:
# # {exampler no. 1} + {exampler no. 2} + {exampler no. 3}... + {final_suffix}

# # Each exampler has the following format:
# # {inputs_prefix} + {inputs} + {x_y_delimiter} + {targets_prefix} + {targets} +
# # {example_separator}
# # """

# # # riga 4450
# # FewShotPattern(
# #     inputs="{text}\n\nWhat is a good title for this?",
# #     targets="{title}",
# #     inputs_prefix="Problem: ",
# #     targets_prefix="Title: ",
# #     x_y_delimiter="\n\n",
# #     example_separator="\n\n\n",
# #     in_template_mix=False),

# ex_test_1="This refers to the manufacture, use and disposal of lifestyle goods such as electronics and clothing. Key sectors of high impact include food, housing and transport. In these sectors, wafer, energy and waste can be considered as crosscutting sectors affecting and affected by almost every lifestyle domain."
# ex_test_1_sdg="SDG 12"

# ex_test_2="The modern definitions of the inchoate offences are well known in the criminal law since the Middle Ages both in European-Continent legal systems and in the English Common Law. They were mostly developed by the case-laws of the Star Chamber Court, which was abolished in 1640. The inchoate offences include three basic offences: Attempt, Conspiracy and Solicitation, but different legal systems defined some other offences as inchoate, such as the Aider and Abettor in Britain after the full validation of the Serious Crimes Act, 2007, c.27, s.44. In this paper it is argued, that there is a very intensive interaction between the fight against terrorism and inchoate offences. The fight against terrorism effects the definition of inchoate offences, and inchoate offences are used as a major instrument of criminal law in the legal fight against terrorism."
# ex_test_2_sdg="SDG 16"

# ex_test_3="As a result, both the trade and current account deficits increased in 2012. Large current account deficits, despite the slowdown in economic growth, are symptomatic of demand-supply imbalances and a pointer to the urgent need to resolve supply bottlenecks. However, capital flows have been adequate to cover the current account deficit thus far."
# ex_test_3_sdg="SDG 8"


# inputs_prefix="Input: "
# inputs=f"{ex_test_1}\n\nWhat is a good SDG (Sustainable Development Goal) for this?"
# x_y_delimiter="\n\n"
# targets_prefix="Output: "
# targets=ex_test_1_sdg
# example_separator="\n\n\n"

# example1 = f"{inputs_prefix}{inputs}{x_y_delimiter}{targets_prefix}{targets}{example_separator}"

# inputs=f"{ex_test_2}\n\nWhat is a good SDG (Sustainable Development Goal) for this?"
# targets=ex_test_2_sdg
# example2 = f"{inputs_prefix}{inputs}{x_y_delimiter}{targets_prefix}{targets}{example_separator}"

# inputs=f"{ex_test_3}\n\nWhat is a good SDG (Sustainable Development Goal) for this?"
# targets=ex_test_3_sdg
# example3 = f"{inputs_prefix}{inputs}{x_y_delimiter}{targets_prefix}{targets}{example_separator}"

# examples = example1+example2+example3

# print(examples)

In [ ]:
# @title
# print(examples+"Input: "+text.replace('\n','')+"\n\nWhat is a good SDG (Sustainable Development Goal) for this?\n\nOutput: ")

In [ ]:
data_sel = data #.iloc[181:] #.sample(10)

In [ ]:
%%time

results = []
# cnt = 0
cnt = 182

for _, row in data_sel.iterrows():

    text = row['text']
    text_id = row['text_id']
    sdg = row['sdg']

    # template = FEW_SHOT_3+f"Text: {text}\n\nWhat is this text about?\n\n{options_}"
#     template = FEW_SHOT_3+f"""Text: {text}
# Question: What is this text about?
# Answer: """

    # template = FEW_SHOT_3+f"""Text: """+text.replace('\n','')+"""\nClass:"""

    # template = "Please classify the following text based on its alignment with the UN's Sustainable Development Goals (SDGs) outlined in the Agenda 2030. Identify and describe which goals from the Agenda 2030 this text could contribute to achieving. The text is: {text}"

    template = """TASK: Given the text provided, determine its classification in the context of the Sustainable Development Goals Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015. Identify the most suitable category among SDG_1 (End poverty in all its forms everywhere), \
SDG_2 (End hunger, achieve food security and improved nutrition and promote sustainable agriculture), \
SDG_3 (Ensure healthy lives and promote well-being for all at all ages), \
SDG_4 (Ensure inclusive and equitable quality education and promote lifelong learning opportunities for all), \
SDG_5 (Achieve gender equality and empower all women and girls), \
SDG_6 (Ensure availability and sustainable management of water and sanitation for all), \
SDG_7 (Ensure access to affordable, reliable, sustainable and modern energy for all), \
SDG_8 (Promote sustained, inclusive and sustainable economic growth, full and productive employment and decent work for all), \
SDG_9 (Build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation), \
SDG_10 (Reduce inequality within and among countries), \
SDG_11 (Make cities and human settlements inclusive, safe, resilient and sustainable), \
SDG_12 (Ensure sustainable consumption and production patterns), \
SDG_13 (Take urgent action to combat climate change and its impacts), \
SDG_14 (Conserve and sustainably use the oceans, seas and marine resources for sustainable development), \
SDG_15 (Protect, restore and promote sustainable use of terrestrial ecosystems, sustainably manage forests, combat desertification, halt and reverse land degradation, and halt biodiversity loss), \
SDG_16 (Promote peaceful and inclusive societies for sustainable development, provide access to justice for all and build effective, accountable and inclusive institutions at all levels), \
SDG_0 (Other).

TEXT: """+text.replace('\n','')+"""
CLASSIFICATION:"""

    #print("#########################################################################################")
    #print(template)

    # template = template_part1 + f"\"{text}\"\n Class name: "

    # template = examples+"Input: "+text.replace('\n','')+"\n\nWhat is a good SDG (Sustainable Development Goal) for this?\n\nOutput: "

    #print(template)
    try:
      input_ids = tokenizer(template, return_tensors="pt").input_ids.to("cuda")
      outputs = model.generate(input_ids)
      results.append([
          text_id,
          sdg,
          tokenizer.decode(outputs[0])
      ])

      result_extended = (cnt, text_id, sdg, tokenizer.decode(outputs[0]))

      # print(result_extended)

      with open(output_data_path+f'/sdg_classification_T5_13B_{cnt}.pkl', 'wb') as f:
          pickle.dump(result_extended, f)

      print((cnt, text_id, sdg, tokenizer.decode(outputs[0])))

    except:
      print((cnt, text_id, sdg, "ERROR"))

    cnt+=1

    # if cnt%100==0:
    #    print(cnt)

    # if cnt==10:
    #     break

#print(template)

(182, 571, 3, '<pad> SDG_3</s>')
(183, 4948, 3, '<pad> SDG_3</s>')
(184, 5101, 3, '<pad> SDG_3</s>')


Token indices sequence length is longer than the specified maximum sequence length for this model (557 > 512). Running this sequence through the model will result in indexing errors


(185, 2852, 3, '<pad> SDG_3</s>')
(186, 6575, 3, '<pad> SDG_3</s>')
(187, 1303, 3, '<pad> SDG_3</s>')
(188, 6582, 3, '<pad> SDG_3</s>')
(189, 5625, 3, '<pad> SDG_3</s>')
(190, 3913, 3, '<pad> SDG_3</s>')
(191, 1122, 3, '<pad> SDG_3</s>')
(192, 6305, 3, '<pad> SDG_3</s>')
(193, 1655, 3, '<pad> SDG_3</s>')
(194, 1028, 3, '<pad> SDG_3</s>')
(195, 6378, 3, '<pad> SDG_3</s>')
(196, 1869, 3, '<pad> SDG_3</s>')
(197, 1372, 3, '<pad> SDG_3</s>')
(198, 3450, 3, '<pad> SDG_3</s>')
(199, 1807, 3, '<pad> SDG_3</s>')
(200, 1124, 3, '<pad> SDG_3</s>')
(201, 1131, 3, '<pad> SDG_3</s>')
(202, 4254, 3, '<pad> SDG_3</s>')
(203, 5220, 3, '<pad> SDG_3</s>')


In [ ]:
# results

In [ ]:
# with open(output_data_path+f'/sdg_classification_T5_13B_{cnt}.pkl', 'wb') as f:
#     pickle.dump(result_extended, f)

In [ ]:
results_processed = pd.DataFrame(results, columns=['id', 'true_label', 'predicted_label'])
results_processed

In [ ]:
# results_processed['label'] = results_processed['predicted_label'].str.extract(r'SDG (\d+)')
results_processed['label'] = results_processed['predicted_label'].str.replace('<pad> ', '')
results_processed['label'] = results_processed['label'].str.replace('</s>', '')

In [ ]:
results_processed.groupby('label').count()

In [ ]:
# label_list = ['dummy', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']

# results_processed = results_processed[results_processed.label.str.len() == 1]
# results_processed['label'] = results_processed['label'].apply(lambda x : label_list.index(x))
# results_processed

In [ ]:
results_processed['label'] = results_processed['label'].str.replace("_", " ")
results_processed['label'] = results_processed['label'].str.extract(r'SDG (\d+)')

In [ ]:
final = data_sel.merge(results_processed, left_index=True, right_index=True)
final

In [ ]:
final.to_excel(outfilename)

In [ ]:
final = final[~final.label.isna()]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_true = final.sdg
y_pred = final.label
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

In [ ]:
classification_report_filename = outdir+f'sdg_classification_T5_1020-texts_classification_report.xlsx'
classification_report_filename

In [ ]:
report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df.to_excel(classification_report_filename)

In [ ]:
test = pd.read_excel(classification_report_filename)
test